# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([25, 78, 79, 22, 26, 37, 38, 31, 76, 18, 18, 18, 72, 79, 22, 22, 56,
       31, 12, 79, 69, 17, 13, 17, 37, 73, 31, 79, 38, 37, 31, 79, 13, 13,
       31, 79, 13, 17, 55, 37,  8, 31, 37, 24, 37, 38, 56, 31, 33, 51, 78,
       79, 22, 22, 56, 31, 12, 79, 69, 17, 13, 56, 31, 17, 73, 31, 33, 51,
       78, 79, 22, 22, 56, 31, 17, 51, 31, 17, 26, 73, 31, 63, 59, 51, 18,
       59, 79, 56, 11, 18, 18,  0, 24, 37, 38, 56, 26, 78, 17, 51])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size, -1)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x) # initialize a numpy array of zeroes with the same shape as x
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[25 78 79 22 26 37 38 31 76 18]
 [73 63 51 31 26 78 79 26 31 79]
 [37 51 40 31 63 38 31 79 31 12]
 [73 31 26 78 37 31 39 78 17 37]
 [31 73 79 59 31 78 37 38 31 26]
 [39 33 73 73 17 63 51 31 79 51]
 [31 57 51 51 79 31 78 79 40 31]
 [48 77 13 63 51 73 55 56 11 31]]

y
 [[78 79 22 26 37 38 31 76 18 18]
 [63 51 31 26 78 79 26 31 79 26]
 [51 40 31 63 38 31 79 31 12 63]
 [31 26 78 37 31 39 78 17 37 12]
 [73 79 59 31 78 37 38 31 26 37]
 [33 73 73 17 63 51 31 79 51 40]
 [57 51 51 79 31 78 79 40 31 73]
 [77 13 63 51 73 55 56 11 31  4]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [23]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        
        # Define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                           dropout=drop_prob, batch_first=True)
        
        # Define a dropout layer
        self.dropout = nn.Dropout(p=self.drop_prob)
        
        # define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_out, hidden = self.lstm(x, hidden)
        
        # pass through a dropout layer
        out = self.dropout(r_out)
        
        # stack up lstm outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        # put x through the fully connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [30]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [31]:
## TODO: set you model hyperparameters
# define and print the net
n_hidden=64
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 64, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=64, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [32]:
batch_size = 8
seq_length = 50
n_epochs = 5 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 4.3290... Val Loss: 4.3138
Epoch: 1/5... Step: 20... Loss: 3.8362... Val Loss: 3.7133
Epoch: 1/5... Step: 30... Loss: 3.3957... Val Loss: 3.2307
Epoch: 1/5... Step: 40... Loss: 3.2530... Val Loss: 3.1533
Epoch: 1/5... Step: 50... Loss: 3.1234... Val Loss: 3.1424
Epoch: 1/5... Step: 60... Loss: 3.1815... Val Loss: 3.1362
Epoch: 1/5... Step: 70... Loss: 3.1109... Val Loss: 3.1382
Epoch: 1/5... Step: 80... Loss: 3.1319... Val Loss: 3.1361
Epoch: 1/5... Step: 90... Loss: 3.1849... Val Loss: 3.1328
Epoch: 1/5... Step: 100... Loss: 3.1467... Val Loss: 3.1320
Epoch: 1/5... Step: 110... Loss: 3.1756... Val Loss: 3.1301
Epoch: 1/5... Step: 120... Loss: 3.1479... Val Loss: 3.1301
Epoch: 1/5... Step: 130... Loss: 3.1265... Val Loss: 3.1297
Epoch: 1/5... Step: 140... Loss: 3.1447... Val Loss: 3.1275
Epoch: 1/5... Step: 150... Loss: 3.2643... Val Loss: 3.1275
Epoch: 1/5... Step: 160... Loss: 3.1070... Val Loss: 3.1263
Epoch: 1/5... Step: 170... Loss: 3.0794... Val Lo

Epoch: 1/5... Step: 1380... Loss: 2.4362... Val Loss: 2.4575
Epoch: 1/5... Step: 1390... Loss: 2.4404... Val Loss: 2.4553
Epoch: 1/5... Step: 1400... Loss: 2.4967... Val Loss: 2.4545
Epoch: 1/5... Step: 1410... Loss: 2.4630... Val Loss: 2.4524
Epoch: 1/5... Step: 1420... Loss: 2.5697... Val Loss: 2.4585
Epoch: 1/5... Step: 1430... Loss: 2.5023... Val Loss: 2.4493
Epoch: 1/5... Step: 1440... Loss: 2.5826... Val Loss: 2.4508
Epoch: 1/5... Step: 1450... Loss: 2.5355... Val Loss: 2.4466
Epoch: 1/5... Step: 1460... Loss: 2.4986... Val Loss: 2.4476
Epoch: 1/5... Step: 1470... Loss: 2.5014... Val Loss: 2.4484
Epoch: 1/5... Step: 1480... Loss: 2.4540... Val Loss: 2.4420
Epoch: 1/5... Step: 1490... Loss: 2.6215... Val Loss: 2.4380
Epoch: 1/5... Step: 1500... Loss: 2.4263... Val Loss: 2.4367
Epoch: 1/5... Step: 1510... Loss: 2.5609... Val Loss: 2.4394
Epoch: 1/5... Step: 1520... Loss: 2.6012... Val Loss: 2.4337
Epoch: 1/5... Step: 1530... Loss: 2.5315... Val Loss: 2.4330
Epoch: 1/5... Step: 1540

Epoch: 1/5... Step: 2730... Loss: 2.2771... Val Loss: 2.2790
Epoch: 1/5... Step: 2740... Loss: 2.3254... Val Loss: 2.2828
Epoch: 1/5... Step: 2750... Loss: 2.4316... Val Loss: 2.2754
Epoch: 1/5... Step: 2760... Loss: 2.3730... Val Loss: 2.2733
Epoch: 1/5... Step: 2770... Loss: 2.4340... Val Loss: 2.2703
Epoch: 1/5... Step: 2780... Loss: 2.4513... Val Loss: 2.2700
Epoch: 1/5... Step: 2790... Loss: 2.4187... Val Loss: 2.2729
Epoch: 1/5... Step: 2800... Loss: 2.5200... Val Loss: 2.2717
Epoch: 1/5... Step: 2810... Loss: 2.3403... Val Loss: 2.2696
Epoch: 1/5... Step: 2820... Loss: 2.3856... Val Loss: 2.2686
Epoch: 1/5... Step: 2830... Loss: 2.3391... Val Loss: 2.2679
Epoch: 1/5... Step: 2840... Loss: 2.4306... Val Loss: 2.2725
Epoch: 1/5... Step: 2850... Loss: 2.3827... Val Loss: 2.2671
Epoch: 1/5... Step: 2860... Loss: 2.1895... Val Loss: 2.2682
Epoch: 1/5... Step: 2870... Loss: 2.4517... Val Loss: 2.2698
Epoch: 1/5... Step: 2880... Loss: 2.2794... Val Loss: 2.2733
Epoch: 1/5... Step: 2890

Epoch: 1/5... Step: 4080... Loss: 2.3441... Val Loss: 2.1719
Epoch: 1/5... Step: 4090... Loss: 2.1717... Val Loss: 2.1724
Epoch: 1/5... Step: 4100... Loss: 2.1103... Val Loss: 2.1702
Epoch: 1/5... Step: 4110... Loss: 2.2720... Val Loss: 2.1714
Epoch: 1/5... Step: 4120... Loss: 2.1942... Val Loss: 2.1677
Epoch: 1/5... Step: 4130... Loss: 2.2027... Val Loss: 2.1661
Epoch: 1/5... Step: 4140... Loss: 2.3383... Val Loss: 2.1681
Epoch: 1/5... Step: 4150... Loss: 2.2301... Val Loss: 2.1651
Epoch: 1/5... Step: 4160... Loss: 2.2576... Val Loss: 2.1648
Epoch: 1/5... Step: 4170... Loss: 2.3583... Val Loss: 2.1624
Epoch: 1/5... Step: 4180... Loss: 2.2737... Val Loss: 2.1642
Epoch: 1/5... Step: 4190... Loss: 2.1560... Val Loss: 2.1664
Epoch: 1/5... Step: 4200... Loss: 2.2694... Val Loss: 2.1655
Epoch: 1/5... Step: 4210... Loss: 2.1754... Val Loss: 2.1641
Epoch: 1/5... Step: 4220... Loss: 2.2432... Val Loss: 2.1620
Epoch: 1/5... Step: 4230... Loss: 2.2114... Val Loss: 2.1614
Epoch: 1/5... Step: 4240

Epoch: 2/5... Step: 5430... Loss: 2.0481... Val Loss: 2.0872
Epoch: 2/5... Step: 5440... Loss: 2.1228... Val Loss: 2.0929
Epoch: 2/5... Step: 5450... Loss: 2.2846... Val Loss: 2.0864
Epoch: 2/5... Step: 5460... Loss: 2.2577... Val Loss: 2.0847
Epoch: 2/5... Step: 5470... Loss: 2.0997... Val Loss: 2.0881
Epoch: 2/5... Step: 5480... Loss: 2.1463... Val Loss: 2.0868
Epoch: 2/5... Step: 5490... Loss: 2.1479... Val Loss: 2.0854
Epoch: 2/5... Step: 5500... Loss: 2.0523... Val Loss: 2.0905
Epoch: 2/5... Step: 5510... Loss: 2.1807... Val Loss: 2.0873
Epoch: 2/5... Step: 5520... Loss: 2.0736... Val Loss: 2.0892
Epoch: 2/5... Step: 5530... Loss: 2.4258... Val Loss: 2.0881
Epoch: 2/5... Step: 5540... Loss: 2.2524... Val Loss: 2.0865
Epoch: 2/5... Step: 5550... Loss: 2.1850... Val Loss: 2.0847
Epoch: 2/5... Step: 5560... Loss: 2.2977... Val Loss: 2.0834
Epoch: 2/5... Step: 5570... Loss: 2.1820... Val Loss: 2.0827
Epoch: 2/5... Step: 5580... Loss: 2.1638... Val Loss: 2.0813
Epoch: 2/5... Step: 5590

...

Epoch: 4/5... Step: 16130... Loss: 1.8355... Val Loss: 1.8397
Epoch: 4/5... Step: 16140... Loss: 1.9684... Val Loss: 1.8415
Epoch: 4/5... Step: 16150... Loss: 2.0852... Val Loss: 1.8466
Epoch: 4/5... Step: 16160... Loss: 1.9567... Val Loss: 1.8434
Epoch: 4/5... Step: 16170... Loss: 1.8883... Val Loss: 1.8397
Epoch: 4/5... Step: 16180... Loss: 1.9722... Val Loss: 1.8404
Epoch: 4/5... Step: 16190... Loss: 2.0722... Val Loss: 1.8384
Epoch: 4/5... Step: 16200... Loss: 2.0075... Val Loss: 1.8417
Epoch: 4/5... Step: 16210... Loss: 2.0293... Val Loss: 1.8425
Epoch: 4/5... Step: 16220... Loss: 1.9761... Val Loss: 1.8428
Epoch: 4/5... Step: 16230... Loss: 1.9117... Val Loss: 1.8413
Epoch: 4/5... Step: 16240... Loss: 1.8147... Val Loss: 1.8424
Epoch: 4/5... Step: 16250... Loss: 2.1218... Val Loss: 1.8432
Epoch: 4/5... Step: 16260... Loss: 1.8481... Val Loss: 1.8402
Epoch: 4/5... Step: 16270... Loss: 1.9748... Val Loss: 1.8378
Epoch: 4/5... Step: 16280... Loss: 1.8868... Val Loss: 1.8399
Epoch: 4

Epoch: 4/5... Step: 17460... Loss: 1.9603... Val Loss: 1.8341
Epoch: 4/5... Step: 17470... Loss: 2.0282... Val Loss: 1.8318
Epoch: 4/5... Step: 17480... Loss: 2.0658... Val Loss: 1.8316
Epoch: 4/5... Step: 17490... Loss: 1.8034... Val Loss: 1.8336
Epoch: 4/5... Step: 17500... Loss: 1.8362... Val Loss: 1.8336
Epoch: 4/5... Step: 17510... Loss: 1.8044... Val Loss: 1.8331
Epoch: 4/5... Step: 17520... Loss: 1.9299... Val Loss: 1.8322
Epoch: 4/5... Step: 17530... Loss: 1.9264... Val Loss: 1.8296
Epoch: 4/5... Step: 17540... Loss: 1.9817... Val Loss: 1.8315
Epoch: 4/5... Step: 17550... Loss: 1.9511... Val Loss: 1.8324
Epoch: 4/5... Step: 17560... Loss: 1.9732... Val Loss: 1.8336
Epoch: 4/5... Step: 17570... Loss: 1.9608... Val Loss: 1.8336
Epoch: 4/5... Step: 17580... Loss: 1.9986... Val Loss: 1.8349
Epoch: 4/5... Step: 17590... Loss: 1.9553... Val Loss: 1.8328
Epoch: 4/5... Step: 17600... Loss: 1.9447... Val Loss: 1.8346
Epoch: 4/5... Step: 17610... Loss: 1.9636... Val Loss: 1.8337
Epoch: 4

Epoch: 5/5... Step: 18790... Loss: 1.9367... Val Loss: 1.8228
Epoch: 5/5... Step: 18800... Loss: 1.9706... Val Loss: 1.8191
Epoch: 5/5... Step: 18810... Loss: 1.9495... Val Loss: 1.8205
Epoch: 5/5... Step: 18820... Loss: 1.8456... Val Loss: 1.8203
Epoch: 5/5... Step: 18830... Loss: 1.9296... Val Loss: 1.8209
Epoch: 5/5... Step: 18840... Loss: 1.8187... Val Loss: 1.8248
Epoch: 5/5... Step: 18850... Loss: 2.0213... Val Loss: 1.8212
Epoch: 5/5... Step: 18860... Loss: 1.8402... Val Loss: 1.8206
Epoch: 5/5... Step: 18870... Loss: 2.0035... Val Loss: 1.8229
Epoch: 5/5... Step: 18880... Loss: 1.8701... Val Loss: 1.8203
Epoch: 5/5... Step: 18890... Loss: 1.8794... Val Loss: 1.8190
Epoch: 5/5... Step: 18900... Loss: 1.8786... Val Loss: 1.8221
Epoch: 5/5... Step: 18910... Loss: 1.8888... Val Loss: 1.8213
Epoch: 5/5... Step: 18920... Loss: 1.8828... Val Loss: 1.8194
Epoch: 5/5... Step: 18930... Loss: 1.8894... Val Loss: 1.8186
Epoch: 5/5... Step: 18940... Loss: 1.8292... Val Loss: 1.8191
Epoch: 5

Epoch: 5/5... Step: 20120... Loss: 2.0749... Val Loss: 1.8184
Epoch: 5/5... Step: 20130... Loss: 1.7415... Val Loss: 1.8188
Epoch: 5/5... Step: 20140... Loss: 1.8559... Val Loss: 1.8169
Epoch: 5/5... Step: 20150... Loss: 1.8893... Val Loss: 1.8143
Epoch: 5/5... Step: 20160... Loss: 1.6423... Val Loss: 1.8133
Epoch: 5/5... Step: 20170... Loss: 1.8412... Val Loss: 1.8137
Epoch: 5/5... Step: 20180... Loss: 2.0186... Val Loss: 1.8133
Epoch: 5/5... Step: 20190... Loss: 1.9022... Val Loss: 1.8098
Epoch: 5/5... Step: 20200... Loss: 1.8213... Val Loss: 1.8102
Epoch: 5/5... Step: 20210... Loss: 2.0052... Val Loss: 1.8137
Epoch: 5/5... Step: 20220... Loss: 1.9111... Val Loss: 1.8154
Epoch: 5/5... Step: 20230... Loss: 1.9923... Val Loss: 1.8129
Epoch: 5/5... Step: 20240... Loss: 1.8614... Val Loss: 1.8149
Epoch: 5/5... Step: 20250... Loss: 1.8262... Val Loss: 1.8141
Epoch: 5/5... Step: 20260... Loss: 1.8759... Val Loss: 1.8195
Epoch: 5/5... Step: 20270... Loss: 1.9128... Val Loss: 1.8144
Epoch: 5

Epoch: 5/5... Step: 21450... Loss: 1.9879... Val Loss: 1.8093
Epoch: 5/5... Step: 21460... Loss: 1.8069... Val Loss: 1.8109
Epoch: 5/5... Step: 21470... Loss: 1.6783... Val Loss: 1.8056
Epoch: 5/5... Step: 21480... Loss: 1.9785... Val Loss: 1.8039
Epoch: 5/5... Step: 21490... Loss: 1.9872... Val Loss: 1.8045
Epoch: 5/5... Step: 21500... Loss: 1.8917... Val Loss: 1.8063
Epoch: 5/5... Step: 21510... Loss: 1.8453... Val Loss: 1.8092
Epoch: 5/5... Step: 21520... Loss: 1.9266... Val Loss: 1.8085
Epoch: 5/5... Step: 21530... Loss: 1.9430... Val Loss: 1.8098
Epoch: 5/5... Step: 21540... Loss: 1.8707... Val Loss: 1.8078
Epoch: 5/5... Step: 21550... Loss: 1.9802... Val Loss: 1.8095
Epoch: 5/5... Step: 21560... Loss: 1.8041... Val Loss: 1.8077
Epoch: 5/5... Step: 21570... Loss: 2.1657... Val Loss: 1.8058
Epoch: 5/5... Step: 21580... Loss: 1.9525... Val Loss: 1.8039
Epoch: 5/5... Step: 21590... Loss: 1.8410... Val Loss: 1.8033
Epoch: 5/5... Step: 21600... Loss: 1.7481... Val Loss: 1.8048
Epoch: 5

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [33]:
# change the name, for saving multiple files
model_name = 'rnn_5_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [34]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [35]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [36]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna was the land and and the colfly to she went why
him to the tell, and to the laditent of the
the tean that said to her and he had
shis tinging at a some, all and that so and terd, and had so had heald as one and a tome her had
was thach they was to he was a thim theind her the her all at the
tild at astant to his hound his
horse taker of him the cheantill. Ho same home. The contarer that when, she had his that whishing. And he corlince or to
though the last to stice of him her she and the love, and wondes had how, and shis, that all whigh she had
heard were thought was his the sord of
a counting and she to sear the tond of his tomling, that with the sered was a sething well at that whining and have
the
stende hindsing that
she his, were a land on to and and all her and the sater. It to
had were
all
him. His
head of
at of the lascor, with a threm with what her thought. And the cantorisity him, to have time a the santer at him and as ofter or him to his wood the theer of stors as, an

## Loading a checkpoint

In [37]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_5_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [38]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said at inded, and the
lost on her the
saiselt, as had a tord a tirled the stince
and take he cas of the stels and had word, and seet out he was the seatious
ould on her shing he with a all with hind at a song and himserted were and tenting on
her and than the lidting ous and who to thing a sone was tire, the light hirds to the word oucing the loth his headed only her sersess hy his tince he caring out
of she wister, shoold, the them that there him to with the shall thering the ciltarithing to the wonlition, whither were the piting of the corlad as as and
and thought his so and had a this at asked of when, the carse where, with alr of her were to thit alreed on his
comengening the shale to he was with
his with the lence, and to the thought he tasking, and himsenn. It had seen, with stire of and wathelf, his stiled him that the palt had had to his stead to him.

"All and have to
at
to were was at the the was on and so watter, and askenting."

"Yes'le heard it
say thin an ofter